In [1]:
# Setting up the environment.
import numpy as np
import pandas as pd
from scipy import stats

In [20]:
# Load the data from JHU
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
df_d = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [21]:
df.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,133,133,135,135,136,136,136,136,136,136
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,11,11,13,13,13,14,14,14,14,14


In [22]:
US = df[(df["Country_Region"] == "US")]
US_d = df_d[(df_d["Country_Region"] == "US")]

In [23]:
US.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,133,133,135,135,136,136,136,136,136,136
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,11,11,13,13,13,14,14,14,14,14


In [24]:
dateColumns = US.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2' ,'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1)
dates = dateColumns.columns
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20',
       '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20',
       '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20',
       '4/13/20', '4/14/20', '4/15

In [25]:
US_states = US.groupby(['Province_State']).sum()
US_states = US_states.loc[:, '1/22/20':]
US_states = US_states.sort_values(by='Province_State', ascending=False)
US_states.head(5)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Wyoming,0,0,0,0,0,0,0,0,0,0,...,275,282,287,296,305,309,313,317,443,447
Wisconsin,0,0,0,0,0,0,0,0,0,0,...,3428,3555,3721,3875,4053,4199,4346,4499,4625,4845
West Virginia,0,0,0,0,0,0,0,0,0,0,...,611,640,702,728,775,785,890,902,908,939
Washington,1,1,1,1,1,1,1,1,1,1,...,10635,10799,10942,11057,11517,11776,11978,12114,12563,12051
Virginia,0,0,0,0,0,0,0,0,0,0,...,5747,6182,6500,6889,7491,8053,8542,8990,9630,10266


In [26]:
US_states_d = US_d.groupby(['Province_State']).sum()
US_states_d = US_states_d.loc[:, '1/22/20':]
US_states_d = US_states_d.sort_values(by='Province_State', ascending=False)

In [27]:
US_states_d.head(3)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Wyoming,0,0,0,0,0,0,0,0,0,0,...,1,1,1,2,2,2,2,2,2,6
Wisconsin,0,0,0,0,0,0,0,0,0,0,...,154,170,183,197,206,212,220,230,244,248
West Virginia,0,0,0,0,0,0,0,0,0,0,...,8,9,10,12,16,16,18,24,26,27


In [28]:
def daily_DF(df):
    # this cell by Leaf Petersen
    # Copy the data frame
    dfdaily=df.copy()

    # We are on the first iteration
    first=True
    # For every column
    for column in df:
       # If this is the first iteration
       if first:
         # Save the first column
         prev = column
         # Copy over the first column unchanged
         dfdaily[column]=df[column]
         # All subsequent iterations are not the first
         first=False
         # Skip the rest of the loop
         continue
       # We are not the first iteration, so keep going
       # Set the new column contents to the difference of the
       # previous column and the current column
       dfdaily[column] = df[column]-df[prev]
       # The current column is the previous column for the next iteration
       prev = column
        
#     # We only want the totals
#     dfdaily = dfdaily.sum().to_frame()
#     dfdaily = dfdaily.reset_index()
#     # Negatives into positives
#     dfdaily[0] = dfdaily[0].abs()
    
    return(dfdaily)

In [29]:
US_States_daily = daily_DF(US_states)
US_States_daily_d = daily_DF(US_states_d)

In [30]:
US_States_daily_d.head(10)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Wyoming,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,4
Wisconsin,0,0,0,0,0,0,0,0,0,0,...,10,16,13,14,9,6,8,10,14,4
West Virginia,0,0,0,0,0,0,0,0,0,0,...,2,1,1,2,4,0,2,6,2,1
Washington,0,0,0,0,0,0,0,0,0,0,...,7,17,22,27,24,10,23,7,21,-17
Virginia,0,0,0,0,0,0,0,0,0,0,...,8,5,41,13,23,27,19,23,24,25
Virgin Islands,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
Vermont,0,0,0,0,0,0,0,0,0,0,...,1,1,0,4,2,3,0,0,2,0
Utah,0,0,0,0,0,0,0,0,0,0,...,-5,5,2,0,3,2,2,1,4,0
Texas,0,0,0,0,0,0,0,0,0,0,...,22,37,33,39,37,25,14,17,26,23


In [31]:
values = US_States_daily[:].replace(0, np.nan)
values.tail(25)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Maryland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,711.0,536.0,560.0,752.0,788.0,754.0,521.0,837.0,509.0,582.0
Maine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.0,37.0,35.0,26.0,31.0,20.0,20.0,8.0,13.0,19.0
Louisiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,421.0,502.0,433.0,581.0,586.0,462.0,348.0,595.0,331.0,404.0
Kentucky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.0,30.0,162.0,225.0,87.0,185.0,253.0,90.0,154.0,174.0
Kansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,51.0,63.0,111.0,115.0,91.0,84.0,143.0,116.0,167.0
Iowa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123.0,189.0,96.0,146.0,191.0,181.0,389.0,257.0,482.0,107.0
Indiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,431.0,168.0,433.0,582.0,612.0,487.0,570.0,477.0,409.0,341.0
Illinois,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1173.0,1223.0,1345.0,1141.0,1844.0,1582.0,1197.0,1156.0,1546.0,2048.0
Idaho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.0,38.0,9.0,114.0,22.0,46.0,13.0,4.0,64.0,30.0


In [34]:
import plotly.graph_objects as go
import datetime
import numpy as np

z=values
x=dates
y=values.index #the sates

d=US_States_daily_d #deaths for the tooltip:.3f
f=US_States_daily_d
customdata = np.dstack((d, f))

fig = go.Figure(data=go.Heatmap(
        z=z,
        x=x,
        y=y,
        customdata=np.dstack((d, f)),
        hovertemplate='State: %{y}<br>Confirmed: <b> %{z:,.0f} </b> <br>Deaths: <b> %{customdata[0]:,.0f} </b> <extra></extra>',
        colorscale='matter',
        xgap=1,
        ygap=12,
        zmin=0,
        zmid=2500,
        zmax=5000,
        colorbar=dict(thickness=10, title='Confirmed <br>_', separatethousands=True),
        connectgaps=False
               )
        )

fig.update_layout(
        title='Daily cases per state',
        title_x=0.5,
        xaxis_nticks=18,
        yaxis_nticks=64,
        height=1800,
        paper_bgcolor='white', 
        plot_bgcolor='#ffffe6',
        hovermode="x unified",
        hoverlabel=dict(
                bgcolor="white", 
                font_size=16, 
                font_family="Arial",
                bordercolor='#b3b3b3'),
        dragmode=False) #disable panning so mobile apps can receive pinch to zoom events
    
fig.update_xaxes(side='top', fixedrange=True, 
                 showgrid=False,
                 spikethickness=0.4,
                 spikedash='solid',)

fig.update_yaxes(gridcolor='#ffe6cc')

fig.show(displayModeBar=False)
    
import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)